<!-- <img src="media/titlepage.png" width="100%"> -->


# Tutorial on Trixi.jl at ICOSAHOM 2021

## Brief introduction to Julia

### Michael Schlottke-Lakemper, Hendrik Ranocha 


- Follow along at https://git.io/JcDM1
- Launch MyBinder at https://tinyurl.com/5spztds3
<!--   (https://mybinder.org/v2/gh/trixi-framework/tutorial-2021-icosahom/HEAD?filepath=introduction_to_julia.ipynb) -->

# Brief introduction to Julia

[Julia](https://julialang.org) is a modern high-level programming language developed specifically with scientific computing in mind. We will briefly introduce Julia and demonstrate some of its design principles to help you getting started with the tutorial on [Trixi.jl](https://github.com/trixi-framework/Trixi.jl), our framework of high-order methods for hyperbolic PDEs written in Julia. This introduction is aimed at researchers in numerical analysis with previous programming experience.


## Further information on running this notebook

This introduction is available as a Jupyter notebook at https://github.com/trixi-framework/tutorial-2021-icosahom, including information how to set up everything. For more information about Trixi and how to use it, please visit [Trixi on GitHub](https://github.com/trixi-framework/Trixi.jl) or refer to the [official documentation](https://trixi-framework.github.io/Trixi.jl/stable/).

This notebook was set up and tested with Julia v1.6.1 but may also work with other (newer) versions.

*Note:* If you change a variable in a later cell and then re-execute an earlier cell, the results might change unexpectedly. Thus if in doubt, re-run the entire notebook *in order*. The reason is that all cells in a Jupyter notebooks share a common variable space.

*Note:* This notebook is tested using Chromium. Most parts should also work for other browsers such as Firefox, but the videos used in the last demonstrations might not be displayed correctly.


## Authors and license

This material is distributed by Michael Schlottke-Lakemper and Hendrik Ranocha under the MIT license. It is inspired by and partially derived from the talks
- [Robin Deits (2020), Intro to Julia Programming Language with Detroit Tech Watch](https://www.youtube.com/watch?v=qLO-yaUkLKE)
- [Hendrik Ranocha (2021), Introduction to Julia and Trixi, a numerical simulation framework for hyperbolic PDEs](https://github.com/trixi-framework/talk-2021-Introduction_to_Julia_and_Trixi)

In [ ]:
# Install all dependencies used in this talk
using Pkg
Pkg.activate(".")
Pkg.instantiate()

In [ ]:
ENV["COLUMNS"] = 100 # display width

# Introduction to Julia

- [julialang.org](https://julialang.org)
- Julia is a high-level language like Python/Matlab with the performance of a fast language like C/C++/Fortran
- Julia is designed for scientific computing...
  - N-dimensional arrays
  - Reproducibility
- ...and valuable for general programming
  - Growing ecosystem of packages
  - Rich type system
- Encourages good software development practices

## Julia at a glance

- First public release in 2012, version 1.0 released in 2018
- Free
  - Julia itself is MIT licensed
  - It bundles some GPLed libraries (which can be disabled if desired)
- Built-in JIT compiler transforms Julia code to native assembly at run time
  - Uses LLVM under the hood
- Garbage collected
- Dynamically typed
- Organized via multiple dispatch

## A brief tour of Julia

### The basics

In [ ]:
# Arithmetic
1 + 2

In [ ]:
# Strings
println("Hello world")

In [ ]:
# Arrays
x = [1, 2, 3]
sum(x)

### Unicode

In [ ]:
# type `\beta` + TAB
β = π / 4
tan(β) ≈ sin(β) / cos(β)

In [ ]:
using LinearAlgebra, Plots, LaTeXStrings
n = 1_000
λ = eigvals(randn(n, n))
scatter(real(λ), imag(λ), aspect_ratio=:equal, legend=nothing,
        xguide=L"\operatorname{Re} \lambda", yguide=L"\operatorname{Im} \lambda")

In [ ]:
# to free some memory on mybinder.org
λ = nothing
GC.gc()

### Functions

In [ ]:
function say_hello(to_whom)
    println("Hello ", to_whom)
end

In [ ]:
say_hello("world")

Functions are generic, so you can pass everything that works (duck typing)

In [ ]:
say_hello([1, 2, 3])

### Types

Everything in Julia has a type

In [ ]:
typeof(1)

In [ ]:
typeof(1.0)

In [ ]:
typeof(π)

In [ ]:
typeof([1, 2, 3])

You can create your own types easily

In [ ]:
struct Person
    name::String
end

alice = Person("Alice")

User-defined types are as efficient as anything built-in

In [ ]:
sizeof(Person) == sizeof(Ptr{String})

### Multiple dispatch

Julia does not use classes to organize nouns (types) and verbs (functions). Instead, multiple dispatch is a central design decision. Thus, the compiler chooses an appropriate method of a given function based on the types of all arguments (not their values!).

For more information, see [Stefan Karpinski's talk at JuliaCon (2019)](https://www.youtube.com/watch?v=kc9HwsxE1OY).

In [ ]:
greet(x, y) = println(x, " greets ", y)

In [ ]:
alice = Person("Alice")
bob = Person("Bob")

greet(alice, bob)

Currently there is only one greet() function, and it will work on `x` and `y` of any type:

In [ ]:
greet(π, "ICOSAHOM participants around the world")

We can use abstract types to organize the behavior of related types:

In [ ]:
abstract type Animal end

struct Cat <: Animal
    name::String
end

We've already defined `greet(x, y)` for any `x` and `y`, but we can add another definition for a more specific set of input types.

We can be as specific or as general as we like with the argument types:

In [ ]:
greet(x::Person, y::Animal) = println(x, " pats ", y)

In [ ]:
greet(x::Cat, y) = println(x, " meows at ", y)

Julia will always pick the *most specific* method that matches the provided function arguments.

In [ ]:
fluffy = Cat("Fluffy")

greet(alice, fluffy)

In [ ]:
greet(fluffy, alice)

In [ ]:
struct Dog <: Animal
    name::String
end

greet(x::Dog, y) = println(x, " barks at ", y)

greet(x::Dog, y::Person) = println("$x licks $y's face")

greet(x::Dog, y::Dog) = println("$x sniffs $y's butt")

In [ ]:
fido = Dog("Fido")
rex = Dog("Rex")

greet(alice, fido)

In [ ]:
greet(fido, fluffy)

In [ ]:
greet(fido, bob)

In [ ]:
greet(fido, rex)

If you want to know which `greet` method will be called for a given set of arguments, you can use `@which` to check:

In [ ]:
@which greet(alice, fido)

You can list all of the methods of a given function with `methods`:

In [ ]:
methods(greet)

You can access docstrings using `?`.

In [ ]:
?methods

Tab completion works in Julia and can print possible signatures of functions.

In [ ]:
greet( # type TAB

## Modules

Modules in Julia are used to organize code into namespaces.

In [ ]:
module MyUsefulModule

export hello

hello()   = println("Hello world")
goodbye() = println("Goodbye world")

end

MyUsefulModule.hello()

The `using` command brings any `export`ed symbols from a module into the current namespace:

In [ ]:
using .MyUsefulModule
hello()

## Using packages

Julia has a built-in package manager called `Pkg`. It handles installing packages and managing all your package environments. 

A package *environment* represents a single set of installed packages. Let's activate the environment for this talk:

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

(this is similar to `source venv/bin/activate` in a Python virtual environment)

We can install a package in our current environment. This will only affect that environment, so we can safely do this without breaking any other Julia projects we might be working on:

In [ ]:
Pkg.add("BenchmarkTools")

The `Project.toml` file gives a concise description of the packages we've added to this environment:

In [ ]:
run(`cat Project.toml`)

The package manager also generates a complete manifest of every package that is installed, including all the transitive dependencies and their versions. You can use this to reproduce a given package environment exactly:

In [ ]:
run(`head Manifest.toml`)

# Bonus features of Julia

## Anything can be a value

Julia has no special rules about what can or cannot be assigned to a variable or passed to a function. 

### Functions are values

A Julia function is a value like any other, so passing functions around and implementing higher-order functions is trivial. This approach is used in several higher order functions of Julia such as `mapreduce`.

In [ ]:
data = randn(10^4)
sum(data) ≈ mapreduce(identity, +, data)

Functions can be inlined, even into standard library code. The compiler heuristics are often good and you can nudge it using `@inline` if necessary. Let's measure the performance of `mapreduce` with out own functions using `@benchmark` from BenchmarkTools:

In [ ]:
using BenchmarkTools

my_identity(x) = x
my_plus(x, y) = x + y
@benchmark mapreduce($my_identity, $my_plus, $data)

If we tell the compiler not to inline our fucntions, the performance will be reduced significantly, of course.

In [ ]:
@noinline my_identity_not_inlined(x) = x
@noinline my_plus_not_inlined(x, y) = x + y
@benchmark mapreduce($my_identity_not_inlined, $my_plus_not_inlined, $data)

In [ ]:
# to free some memory on mybinder.org
data = nothing
GC.gc()

### Types are values

Types can also be passed around as values and bound to variables with no special rules. This makes implementing factories or constructors easy:

In [ ]:
zeros(Float64, 3, 3)

### Macros

A macro is written just like a normal Julia function. The difference is that a macro operates on the *expression* itself, not on its value:

`@show` : print out the *name* of a variable and its value. Great for quick debugging:

In [ ]:
x = 5
@show x

`@time` measure the elapsed time of an expression and return the result of that expression:

In [ ]:
@time sqrt(big(π))

We have seen its sibling `@benchmark` above.